Asynchrone Microservices
------------------------

![](images/Microservices-Messaging.png)

Quelle: Buch Microservices Rezepte
- - -

Das System besteht aus einem Microservice **order**, der eine Bestellung über die Weboberfläche entgegennimmt. 

Die Bestellung schickt der Bestellprozess dann als Record über Kafka an den Microservice für den Versand **shipping** und den Microservice für die Erstellung der Rechnung **invoicing**.

Die Bestellung wird als JSON übertragen. So können der Rechnungs-Microservice und der Versand-Microservice aus der Datenstruktur jeweils die Daten auslesen, die für den jeweiligen Microservice relevant sind.

Der Versand-Microservice und der Rechnungs-Microservice speichern die Informationen aus den Records in ihren eigenen Datenbank-Schemata. Alle Microservices nutzen eine gemeinsame Postgres-Datenbank.



In [1]:
! kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kafka/apache.yaml
! kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kafka/invoicing.yaml
! kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kafka/kafka.yaml
! kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kafka/order.yaml
! kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kafka/postgres.yaml
! kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kafka/shipping.yaml
! kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kafka/zookeeper.yaml
! kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e 's/https:/http:/' -e "s/6443/$(kubectl get service apache --namespace ms-kafka -o=jsonpath='{ .spec.ports[0].nodePort }')/"

service/apache created
deployment.apps/apache created
service/invoicing created
deployment.apps/invoicing created
service/kafka created
deployment.apps/kafka created
service/order created
deployment.apps/order created
service/postgres created
deployment.apps/postgres created
service/shipping created
deployment.apps/shipping created
service/zookeeper created
deployment.apps/zookeeper created
http://192.168.137.100:32180

Was Microservices zusammenspiel verfolgen wird in der Weave Scope Oberfläche.

Mittels den nachfolgenden Befehlen wird die Weave Scope Oberfläche freigeschaltet.

In [2]:
! kubectl get service weave-scope-app -n weave -o yaml | sed 's/ClusterIP/NodePort/' | kubectl apply -f -
! echo $(kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e 's/https:/http:/' -e "s/6443/$(kubectl get service weave-scope-app --namespace weave -o=jsonpath='{ .spec.ports[0].nodePort }')/")'/#!/state/{"topologyId":"containers","topologyOptions":{"containers":{"namespace":["scsesi","ms-kafka"]}}}'

service/weave-scope-app configured
http://192.168.137.100:30230/#!/state/{"topologyId":"containers","topologyOptions":{"containers":{"namespace":["scsesi","ms-kafka"]},"pods":{"namespace":["scsesi"]}}}


- -  -
Nachdem wir ein paar Aufträge erfasst haben, können wir uns die Meldungen im Messaging Server (Kafka) anschauen.


In [7]:
! kubectl exec -n ms-kafka  $(kubectl get po -l app=kafka -n ms-kafka -o=jsonpath='{ .items[0].metadata.name }') -- bash /opt/kafka/bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic order --from-beginning --timeout-ms 2000 

{"id":7,"customer":{"customerId":1,"name":"Wolff","firstname":"Eberhard","email":"eberhard.wolff@gmail.com"},"updated":1572524890165,"shippingAddress":{"street":"","zip":"","city":""},"billingAddress":{"street":"","zip":"","city":""},"orderLine":[{"count":89,"item":{"itemId":3,"name":"iPod","price":42.0}}],"numberOfLines":1}


- - -

Aufräumen

In [8]:
! kubectl delete -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kafka/apache.yaml
! kubectl delete -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kafka/invoicing.yaml
! kubectl delete -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kafka/kafka.yaml
! kubectl delete -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kafka/order.yaml
! kubectl delete -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kafka/postgres.yaml
! kubectl delete -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kafka/shipping.yaml
! kubectl delete -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kafka/zookeeper.yaml

service "apache" deleted
deployment.apps "apache" deleted
service "invoicing" deleted
deployment.apps "invoicing" deleted
service "kafka" deleted
deployment.apps "kafka" deleted
service "order" deleted
deployment.apps "order" deleted
service "postgres" deleted
deployment.apps "postgres" deleted
service "shipping" deleted
deployment.apps "shipping" deleted
service "zookeeper" deleted
deployment.apps "zookeeper" deleted
